## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%tensorflow_version  2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [3]:
# Importing necessary libraries
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.layers import LeakyReLU

Using TensorFlow backend.


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [7]:
import os
os.chdir('/content/drive/My Drive/DogBreed_Classification')
print(os.getcwd())
!ls

/content/drive/My Drive/DogBreed_Classification
labels.csv.zip	sample_submission.csv.zip  test.zip  train.zip


In [8]:
!ls

labels.csv.zip	sample_submission.csv.zip  test.zip  train.zip


In [9]:
print (project_path)

/content/drive/My Drive/DogBreed_Classification/


In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [58]:
labels = pd.read_csv('/content/labels.csv')
labels.head()


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [0]:
img_rows=128
img_cols=128
num_channel=1


### Print the count of each category of Dogs given in the dataset



In [59]:

print('Total count of dogs: ', labels.shape[0])
print('Number of dog Category: ', len(labels.breed.unique()))

Total count of dogs:  10222
Number of dog Category:  120


In [60]:
breed_count = labels['breed'].value_counts()
breed_count.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

### Get one-hot encodings of labels

In [61]:
labels.shape

(10222, 2)

In [0]:
import numpy as np
targets = pd.Series(df_labels['breed'])
one_hot = pd.get_dummies(targets,sparse=True)
one_hot_labels = np.asarray(one_hot)

In [63]:
one_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [98]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i=0
for f,img in tqdm(labels.values):
  train_img = cv2.imread('./train/{}.jpg'.format(f),0)
  label=one_hot_labels[i]
  train_img_resize = cv2.resize(train_img,(img_rows,img_cols))
  x_feature.append(train_img_resize)
  y_feature.append(label)
  i+=1

100%|██████████| 10222/10222 [00:14<00:00, 716.77it/s]


In [99]:
x_train_data = np.array(x_feature,np.float32)/255
print (x_train_data.shape)
x_train_data = np.expand_dims(x_train_data,axis=3)
print (x_train_data.shape)

(10222, 128, 128)
(10222, 128, 128, 1)


In [100]:
y_train_data = np.array(y_feature)
y_train_data.shape

(10222, 120)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [103]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train_data,y_train_data, test_size=0.2, random_state=2)
print (x_train.shape)
print (x_val.shape)

(8177, 128, 128, 1)
(2045, 128, 128, 1)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [104]:
submission = pd.read_csv('/content/sample_submission.csv')
test_img = submission['id']
submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [105]:
test_img.head()

0    000621fb3cbb32d8935728e48679680e
1    00102ee9d8eb90812350685311fe5890
2    0012a730dfa437f5f3613fb75efcd4ce
3    001510bc8570bbeee98c8d80c8a95ec1
4    001a5f3114548acdefa3d4da05474c2e
Name: id, dtype: object

Run the below code to load the test image files in x_test_feature

In [25]:
print (img_cols)

128


In [106]:
x_test = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test.append(img_resize)

100%|██████████| 10357/10357 [00:13<00:00, 766.81it/s]


Normalize the test data and convert it into 4 dimensions

In [107]:
x_test_data = np.array(x_test,np.float32)/255
print (x_test_data.shape)
x_test_data = np.expand_dims(x_test_data,axis =3)
print(x_test_data.shape)

(10357, 128, 128)
(10357, 128, 128, 1)


### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, input_shape = (128,128,1), kernel_size=5)) 
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3))

model.add(tf.keras.layers.Flatten()) 

# fully connected layer
model.add(tf.keras.layers.Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 120, activation = 'softmax'))


### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:

model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [110]:
y_train.shape

(8177, 120)

In [111]:
x_train.shape

(8177, 128, 128, 1)

In [112]:
history = model.fit(x_train, y_train,
                    epochs=10, 
                    validation_data=(x_val, y_val),
                    verbose = 1,
                    batch_size=128)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 15s 2ms/sample - loss: 18.7867 - accuracy: 0.0094 - val_loss: 4.7892 - val_accuracy: 0.0054
Epoch 2/10
8177/8177 [==============================] - 8s 957us/sample - loss: 4.7772 - accuracy: 0.0120 - val_loss: 4.7812 - val_accuracy: 0.0108
Epoch 3/10
8177/8177 [==============================] - 8s 957us/sample - loss: 4.7453 - accuracy: 0.0165 - val_loss: 4.7784 - val_accuracy: 0.0147
Epoch 4/10
8177/8177 [==============================] - 8s 957us/sample - loss: 4.6738 - accuracy: 0.0254 - val_loss: 4.7910 - val_accuracy: 0.0117
Epoch 5/10
8177/8177 [==============================] - 8s 957us/sample - loss: 4.5422 - accuracy: 0.0350 - val_loss: 4.7987 - val_accuracy: 0.0132
Epoch 6/10
8177/8177 [==============================] - 8s 959us/sample - loss: 4.3358 - accuracy: 0.0635 - val_loss: 4.8420 - val_accuracy: 0.0196
Epoch 7/10
8177/8177 [==============================] - 8s 955us

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from keras.layers import ZeroPadding2D

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen=ImageDataGenerator(rescale=1./255)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
training_set=train_datagen.flow(x_train,y=y_train,batch_size=32)
testing_set=test_datagen.flow(x_val,y=y_val,batch_size=32)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
tf.keras.backend.clear_session()
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.ZeroPadding2D((1,1),input_shape=(128,128,1)))
model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.ZeroPadding2D((1,1)))

model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model1.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),activation ='relu')) 
model1.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model1.add(tf.keras.layers.Flatten()) 

# fully connected layer
model1.add(tf.keras.layers.Dense(units=500, activation = 'relu'))
model1.add(tf.keras.layers.Dropout(0.2))

model1.add(tf.keras.layers.Dense(units = 120, activation = 'softmax'))

In [0]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model1_checkpoint=tf.keras.callbacks.ModelCheckpoint('dogs_cnn.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_acc', 
                                                    mode='max', 
                                                    verbose=1)

In [136]:
print (x_train.shape)
print (x_val.shape)

(8177, 128, 128, 1)
(2045, 128, 128, 1)


In [137]:
model1.fit_generator(training_set,
                    epochs=10,
                    steps_per_epoch= 8177//32,  #Number of training images//batch_size
                    validation_data=testing_set,
                    validation_steps = 2045//32, #Number of test images//batch_size
                    callbacks = [model1_checkpoint])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 255 steps, validate for 63 steps
Epoch 1/10
255/255 [==============================] - 12s 46ms/step - loss: 4.7867 - accuracy: 0.0098 - val_loss: 4.7858 - val_accuracy: 0.0119
Epoch 2/10
255/255 [==============================] - 11s 44ms/step - loss: 4.7808 - accuracy: 0.0103 - val_loss: 4.7894 - val_accuracy: 0.0119
Epoch 3/10
255/255 [==============================] - 11s 44ms/step - loss: 4.7790 - accuracy: 0.0093 - val_loss: 4.7898 - val_accuracy: 0.0119
Epoch 4/10
255/255 [==============================] - 11s 44ms/step - loss: 4.7776 - accuracy: 0.0117 - val_loss: 4.7882 - val_accuracy: 0.0119
Epoch 5/10
255/255 [==============================] - 11s 43ms/step - loss: 4.7773 - accuracy: 0.0118 - val_loss: 4.7886 - val_accuracy: 0.0119
Epoch 6/10
255/255 [==============================] - 11s 44ms/step - loss: 4.7765 - accuracy: 0.0125 - val_loss: 4.7890 - val_accuracy: 0.0119
Epoch 7/10
255/255 [=======================

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
project_path1 = "/content/drive/My Drive/Notebooks/"

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= tf.keras.applications.VGG16(weights=(project_path1+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),include_top=False, pooling='avg')

Print the summary of the base_model

In [162]:

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification 

---

layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
base_model = tf.keras.models.Sequential()
base_model.add(tf.keras.layers.Flatten()) 

base_model.add(tf.keras.layers.Dense(units=1024, activation = 'relu'))
base_model.add(tf.keras.layers.Dense(units=256, activation = 'relu'))
base_model.add(tf.keras.layers.Dense(units=120, activation = 'softmax'))

### Make all the layers in the base_model (VGG16) to be non-trainable

In [164]:
for layer in base_model.layers:
  print (layer.name)

flatten_2
dense_5
dense_6
dense_7


In [0]:
for layer in base_model.layers:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
base_model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])


In [166]:
history = base_model.fit(x_train, y_train,
                    epochs=10, 
                    validation_data=(x_val, y_val),
                    verbose = 1,
                    batch_size=128)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 1s 110us/sample - loss: 6.1788 - accuracy: 0.0094 - val_loss: 4.7864 - val_accuracy: 0.0083
Epoch 2/10
8177/8177 [==============================] - 1s 67us/sample - loss: 4.7862 - accuracy: 0.0088 - val_loss: 4.7871 - val_accuracy: 0.0078
Epoch 3/10
8177/8177 [==============================] - 1s 66us/sample - loss: 4.7853 - accuracy: 0.0120 - val_loss: 4.7867 - val_accuracy: 0.0078
Epoch 4/10
8177/8177 [==============================] - 1s 68us/sample - loss: 4.7841 - accuracy: 0.0106 - val_loss: 4.7864 - val_accuracy: 0.0078
Epoch 5/10
8177/8177 [==============================] - 1s 71us/sample - loss: 4.7826 - accuracy: 0.0133 - val_loss: 4.7863 - val_accuracy: 0.0078
Epoch 6/10
8177/8177 [==============================] - 1s 73us/sample - loss: 4.7823 - accuracy: 0.0120 - val_loss: 4.7862 - val_accuracy: 0.0078
Epoch 7/10
8177/8177 [==============================] - 1s 69us/sampl